# 타이타닉 생존자 머신러닝 모델

## 1. 테스트 환경 셋팅

### 라이브러리 불러오기
import pandas as pd 타이타닉 데이터는 표 데이터이기 때문에 pandas를 불러온다.  
from sklearn.model_selection import train_test_split 학습용, 테스트용 분리를 위해   
from sklearn.metrics import accuracy_score 예측이 맞았는지 수치 확인을 위해. 맞춘개수/전체개수. 혼동행렬 기준으로 (TP+TN)/전체값  
from sklearn.preprocessing import LabelEncoder 문자를 숫자로 바꾸기 위해 문자라벨을 숫자로 변환한다. 중복을 제거하여 정렬하고 0부터 차례대로 번호를  부여한다.  
from sklearn.metrics import confusion_matrix 어디서 틀렸는지 알기 위해   
(예측1은 P, 0은 N, 실제와 비교해 맞으면T, 틀리면F)  
- 예측이 1인데 실제를 확인해보니 1=> P가 맞아서 T, TP  
- 예측이 1인데 실제를 확인해보니 0 => P가 틀려서F, FP 
- 예측이 0인데 실제를 확인해보니 0 => N이 맞아서 T, TN 
- 예측이 0인데 실제를 확인해보니 1 => N이 맞아서 F, FN
from sklearn.metrics import precision_score, recall_score
- 정밀도: 내가 1P이라고 예측한 것들 중에서 실제로 1P인 비율. TP/(TP+FP) 값
- 재현율: 실제로 1P인데 예측에서 1P로 맞춘 비율 TP/(TP+FN)값

In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import confusion_matrix
from sklearn.metrics import precision_score, recall_score

## 2. 데이터 불러오고 EDA

### 판다스 import 및 데이터프레임 정의
url을 변수에 담고 url에 있는 csv 파일을 열어서 RAM으로 읽어온다.  
이것을 df 변수에 저장하고 상단 2개 목록만 불러온다.  
만약 csv 파일을 저장하고 싶으면 df.to_csv("titanic.csv", index=False) index번호 안붙임  

In [2]:
import pandas as pd
url = 'https://raw.githubusercontent.com/datasciencedojo/datasets/master/titanic.csv'
df = pd.read_csv(url)
df.head(2)

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C


## 3. 데이터 전처리

### 결측치 처리
fillna 함수를 정의하되 입력 파라미터는 df로 정의  
df.copy() 원본데이터를 복사해서 편집하기 위해  
컬럼값 'Age'는 숫자이다. 빈값을 평균값으로 채워 'Age'에 저장  
컬럼값 'Cabin', 'Embarked'은 문자형임으로 문자 N으로 채운다.  
컬럼값 'Fare'를 숫자0으로 채운다.  

In [3]:
def fillna(df):
    df = df.copy()  # 명시적 복사
    df['Age'] = df['Age'].fillna(df['Age'].mean())
    df['Cabin'] = df['Cabin'].fillna('N')
    df['Embarked'] = df['Embarked'].fillna('N')
    df['Fare'] = df['Fare'].fillna(0)
    return df

### 열삭제 처리
axis=0은 행방향, 1은 열방향. 열방향으로 3개 칼럼을 삭제함.  
이 전처리를 반복해서 쓸 수 있으므로 함수 정의  
df는 로컬변수이므로 안에서만 사용가능  

In [4]:
def drop_features(df):
    return df.drop(['PassengerId', 'Name', 'Ticket'], axis=1)

### 문자형 데이터를 숫자형으로 변환
Cabin에서 문자열의 첫글자만 사용해서 cabin에 저장  
features-숫자로 바꿔야할 컬럼목록이다. sex, embarked는 범주개수가 단순. 반면 cabin은 고유값이 너무 많아서 cabin만 따로 처리함  
features 3개 컬럼을 1개씩 돌려가며 반복.하되 LabelEncoder로 숫자를 부여한다.  
컬럼 1개를 가져와서, 
fit을 써서 값을 불러오고 번호표가 매핑되어 있다. transform으로 번호표로 바꿔준다.  

In [5]:
def format_features(df):
    df = df.copy()
    df['Cabin'] = df['Cabin'].str[:1]
    features = ['Cabin', 'Sex', 'Embarked']
    for feature in features:
        le = LabelEncoder()
        df[feature] = le.fit_transform(df[feature])
    return df


### 함수들 한번에 처리
결측치 처리한 함수, 불필요한 칼럼 제거한 함수, 숫자로 변환한 함수를 묶어서 하나의 함수로 처리  

In [6]:
def transform_features(df):
    df = fillna(df)
    df = drop_features(df)
    df = format_features(df)
    return df

### 4개 모델을 함수로 묶음
두개의 파라미터 입력값이 있는 함수 정의, 함수정의할때는 실행되지 않는다. 함수값을 호출할 때 실행된다.  
y_test는 실제 정답, pred는 예측값, 두개의 값을 입력해서 혼동행렬, 정확도, 정밀도, 재현율 계산  
혼동행렬: TP: 맞게 생존 예측, FP: 틀리게 생존 예측, FN: 놓친 생존자, TN: 맞게 사망 예측  
정확도: 전체개수중 맞춘 개수  
정밀도: 생존으로 예측했던 사람중 실제 생존자수  
재현율: 실제 생존자 중 몇명을 맞췄는가  

In [7]:
def get_clf_eval(y_test, pred):
    confusion = confusion_matrix(y_test, pred)
    accuracy = accuracy_score(y_test, pred)
    precision = precision_score(y_test, pred)
    recall = recall_score(y_test, pred)

    print(confusion)
    print('*'*20)
    print(accuracy, precision, recall)

### 전처리 데이터를 변수로 저장
머신러닝 모델이 학습할 수 있는 데이터  

In [8]:
titanic_df = transform_features(df)

### 입력과 정답을 분리 
Survived 칼럼을 정답값으로 정의하고 따로 y에 저장  
나머지 칼럼을 입력값으로 정의하고 x에 저장  

In [9]:
y_titanic_df = titanic_df['Survived']
X_titanic_df = titanic_df.drop('Survived', axis=1)

## 4. 데이터분할

### 학습용, 테스트용 데이터 나누기
x학습용, x테스트, y학습, y테스트로 나누다.  
입력데이터, 정답 데이터 구분, 학습용과 테스트를 0.2, 즉 20%는 테스트, 나머지 80%는 학습용  
랜덤값 0. 임의값 지정. 이렇게 하지 않으면 돌릴때마다 랜덤값이 다르다.  

In [10]:
X_train, X_test, y_train, y_test = train_test_split(X_titanic_df,
                                                    y_titanic_df,
                                                    test_size=0.2,
                                                    random_state=0 )

## 5. 모델 선택

### 로지스틱 회귀
- klearn.linear_model모듈에서 로지스틱 회귀 클래스를 가져온다.입력 변수(X)를 받아서 특정 클래스에 속할 확률을 예측하고, 확률이 일정 기준(보통 0.5) 이상이면 1, 미만이면 0 등으로 분류한다.  
- lr_clf라는 이름으로 로지스틱 회귀 모델 객체 생성, 최대 반복회수는 2000번. 최적 가중치 찾기 위해  
- fit: 학습 데이터 X_train과 정답 y_train을 이용해 로지스틱 회귀 모델이 최적의 가중치와 편향을 학습하도록 하는 명령  
- 학습되지 않은 X_test데이터가 회귀모델에서 얼마나 잘 예측하는지 해보세요.  
- 4개 모델을 묶은 get_clf_eval함수안에 실제 정답인 y_test와 모델이 옟측한 값을 비교해주세요.  

In [11]:
#로지스틱회귀 분류모델 생성
from sklearn.linear_model import LogisticRegression

lr_clf = LogisticRegression(max_iter=2000)
lr_clf.fit(X_train, y_train)
pred = lr_clf.predict(X_test)

#정확도, 정밀도, 재현율
get_clf_eval(y_test, pred)

[[92 18]
 [16 53]]
********************
0.8100558659217877 0.7464788732394366 0.7681159420289855


## 6. 단순가설의 분류기를 이용한 성능 기준 확인

### 나만의 상자모델 만듬
### 표안에 초기값0, sex값 넣음
- sklearn.base는 모델을 만들때 기본틀을 제공하는 모듈. 
- BaseEstimator는 모델의 뼈대를 제공함. 이를 상속하면 사이킷런 스타일 규칙을 따른다.  
- 사이킷런에서 쓸 수 있는 나만의 빈 상자 모델을 만들었고, 그 안에 함수를 만듬.  
- 함수안에 입력값X, 정답값y를 보고 학습시킨다.  
- 클래스안에서 변수나 함수를 사용할 때 항상 self를 붙여야 한다.  
- 이 코드는 더미 분류기 이다. fit에서 학습이 없고 바로 pass, predict에서 단순규칙만 사용. 일종의 기준점. '시험공부 안하고 찍었을 때 평균점수'를 뜻한다.  
- X.shape[0]는 행수, X.shape[1]는 열수  
- np.zeros(행수, 열수) : 지정한 행,열 표로 만들되 그 안을 0으로 채워라  
- 행수는 X.shape[0]만큼, 열은 1열. X.shape[0] 행 × 1 열의 표 만들어짐. 초기값 0으로 들어가 있음  
- for: X.shape[0]는 X의 행수, range는 0부터 숫자 시퀀스, for i in는 숫자 시퀀스를 하나씩 i 변수에 담아 반복실행  
- if~: X에서 Sex 컬럼, i행 값이 1이면 남성,i행에 0을 넣고, 여성이면 1값을 넣는다.  

In [12]:
from sklearn.base import BaseEstimator
import numpy as np
class MyDummyClassifier(BaseEstimator):
  def fit(self, X, y):
    pass

  def predict(self, X):
    pred = np.zeros((X.shape[0],1))
    for i in range(X.shape[0]): #테스트데이터 갯수 만큼 
      if X['Sex'].iloc[i] == 1:
        pred[i]=0
      else :
        pred[i]=1
    return pred

### 더미 분류기 생성, 학습, 예측
- 더미 분류기 객체 생성  
- fit~: 학습 데이터 X_train과 y_train을 모델에 전달, 학습없이 predict만 호출  
- predict~: X_test를 이용해 모델이 규칙대로 예측한 결과를 my_pred 배열에 저장  
- 실제 정답 y_test, 예측과 비교하여 맞춘비율 정확도 계산  

In [13]:
myclf = MyDummyClassifier()
myclf.fit(X_train, y_train)
my_pred = myclf.predict(X_test)
accuracy_score(y_test, my_pred)

0.7877094972067039

## 7. 모델 평가

### 랜덤포레스트, KNN import
- RandomForestClassifier : 분류(Classification) 문제를 해결하는 모델  
- ensemble = 여러 개의 작은 나무(결정 기준)를 만들어서, 가장 많은 나무가 선택한 답을 최종 예측  
- KNeighborsClassifier = K-최근접 이웃(K-Nearest Neighbors, KNN) 기반의 분류 모델  

### 랜덤포레스트 결정트리개수, KNN 이웃개수 지정하여 학습, 예측
n_estimators 결정트리개수 100개, n_neighbors 이웃개수 5개  
랜덤포레스트, KNN 학습 (학습 입력데이터 features, 정답데이터 target)  
학습용 입력값과 정답값을 각각의 방법으로 학습시킨다.  
테스트용 X값을 각각의 방법으로 예측한다.  

### 결과
로지스틱회귀, 랜덤포레스트, KNN 정확도, 정밀도, 재현율, F1 스코어  
F1 스코어: 정밀도와 재현율 고려한 종합점수  
 

In [14]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier

# 모델 생성 및 학습
rf_clf = RandomForestClassifier(n_estimators=100, random_state=0)
knn_clf = KNeighborsClassifier(n_neighbors=5)

rf_clf.fit(X_train, y_train)
knn_clf.fit(X_train, y_train)

# 예측
rf_pred = rf_clf.predict(X_test)
knn_pred = knn_clf.predict(X_test)

# 평가
print("=" * 40)
print("로지스틱 회귀")
print("=" * 40)
get_clf_eval(y_test, pred)

print("\n" + "=" * 40)
print("랜덤포레스트")
print("=" * 40)
get_clf_eval(y_test, rf_pred)

print("\n" + "=" * 40)
print("KNN")
print("=" * 40)
get_clf_eval(y_test, knn_pred)

로지스틱 회귀
[[92 18]
 [16 53]]
********************
0.8100558659217877 0.7464788732394366 0.7681159420289855

랜덤포레스트
[[99 11]
 [20 49]]
********************
0.8268156424581006 0.8166666666666667 0.7101449275362319

KNN
[[94 16]
 [31 38]]
********************
0.7374301675977654 0.7037037037037037 0.5507246376811594


### 점수의 의미
1에 가까울 수록 성능이 좋다. 모델이 실제 정답과 얼마나 잘 맞았는지를 보여주는 점수  
- Accuracy(정확도): 전체 예측 중 맞춘 비율이 높다. 10문제 중 맞춘 문제 수가 많음  
- Precision(정밀도): 모델이 맞다고 한 것 중 실제로 맞은 비율이 높다. 내가 맞다고 표시한 답이 대부분 맞음  
- Recall(재현율): 실제 맞는 것 중 모델이 맞춘 비율이 높다. 실제 정답을 거의 놓치지 않음  
- F1 Score: Precision과 Recall이 모두 높다. 정확도와 놓치지 않은 정도를 동시에 잘 달성  
- F1 점수 = 0.9 → 모델이 생존/사망을 거의 정확히 맞추고, 동시에 놓치는 경우도 거의 없음  
- F1 점수 = 0.5 → 모델이 정확히 맞춘 것도 적고, 놓치는 경우도 많음  